# Thermodynamic Property Curves: Ag-Cu FCC

This notebook demonstrates the thermodynamic property calculation functions in OpenCALPHAD.jl.

**Corresponds to:** step2.OCM from openCALPHAD (Fortran)

**Contents:**
- Gibbs energy curve with common tangent
- Mixing Gibbs energy (G_mix)
- Mixing enthalpy (H_mix)
- Mixing entropy (S_mix)

In [ ]:
using OpenCALPHAD
using Plots

## 1. Load Database and Set Parameters

In [ ]:
# Load Ag-Cu thermodynamic database
tdb_path = joinpath(@__DIR__, "..", "reftest", "tdb", "agcu.TDB")
db = read_tdb(tdb_path)
fcc = get_phase(db, "FCC_A1")

# Show phase structure
println("Phase: $(fcc.name)")
println("Constituents: $(fcc.constituents)")
println("Sites: $(fcc.sites)")

In [ ]:
# Calculation parameters
T = 1000.0  # Temperature [K]
P = 1e5     # Pressure [Pa]

println("Temperature: $(T) K")
println("Pressure: $(P/1e5) bar")

## 2. Calculate Gibbs Energy vs Composition

In [ ]:
# Scan composition using GridSearchSolver
solver = GridSearchSolver(n_points=101)
scan = scan_composition(fcc, T, db, solver)

# x_Ag = fraction of first constituent (Ag)
x_Ag = scan.x_grid
x_Cu = 1.0 .- x_Ag
G_values = scan.G_values

println("Composition range: x(Ag) = $(x_Ag[1]) to $(x_Ag[end])")
println("Number of points: $(length(x_Ag))")

## 3. Calculate Mixing Properties

Using `thermodynamic_properties()` to get G_mix, H_mix, S_mix in one call.

In [ ]:
# Calculate mixing properties for each composition point
G_mix = Float64[]
H_mix = Float64[]
S_mix = Float64[]

for y in scan.y_values
    props = thermodynamic_properties(fcc, T, y, db; P=P)
    push!(G_mix, props.G_mix)
    push!(H_mix, props.H_mix)
    push!(S_mix, props.S_mix)
end

# Values at x(Ag) = 0.5
idx_mid = findfirst(x -> x >= 0.5, x_Ag)
println("At x(Ag) = 0.5:")
println("  G_mix = $(round(G_mix[idx_mid], digits=1)) J/mol")
println("  H_mix = $(round(H_mix[idx_mid], digits=1)) J/mol")
println("  S_mix = $(round(S_mix[idx_mid], digits=3)) J/(mol·K)")
println("  -T*S_mix = $(round(-T*S_mix[idx_mid], digits=1)) J/mol")

## 4. Find Miscibility Gap

In [ ]:
gap = find_miscibility_gap(fcc, T, db)

if !isnothing(gap)
    println("Miscibility gap found:")
    println("  x1 = $(round(gap.x1, digits=4)) (Cu-rich phase)")
    println("  x2 = $(round(gap.x2, digits=4)) (Ag-rich phase)")
    println("  G_eq = $(round(gap.G1/1000, digits=2)) kJ/mol")
else
    println("No miscibility gap at this temperature")
end

## 5. Calculate Reference Line (Mechanical Mixture)

In [ ]:
# Pure component Gibbs energies
n_sub = length(fcc.sites)
n_const = length(fcc.constituents[1])

# Pure Ag
y_Ag = zeros(n_sub, n_const)
y_Ag[1, 1] = 1.0
for s in 2:n_sub
    y_Ag[s, 1] = 1.0
end
G_Ag_pure = calculate_gibbs_energy(fcc, T, y_Ag, db; P=P)

# Pure Cu
y_Cu = zeros(n_sub, n_const)
y_Cu[1, 2] = 1.0
for s in 2:n_sub
    y_Cu[s, 1] = 1.0
end
G_Cu_pure = calculate_gibbs_energy(fcc, T, y_Cu, db; P=P)

# Reference line: G_ref = x_Ag * G_Ag + x_Cu * G_Cu
G_ref_line = x_Ag .* G_Ag_pure .+ x_Cu .* G_Cu_pure

println("Pure Ag (FCC): G = $(round(G_Ag_pure/1000, digits=2)) kJ/mol")
println("Pure Cu (FCC): G = $(round(G_Cu_pure/1000, digits=2)) kJ/mol")

## 6. Plot Results

In [ ]:
# Panel (a): Gibbs energy with common tangent
pa = plot(x_Ag, G_values ./ 1000,
    xlabel="x(Ag)", ylabel="G [kJ/mol]",
    title="(a) Gibbs Energy",
    label="G(x)", linewidth=2, color=:blue,
    legend=:topright)

plot!(pa, x_Ag, G_ref_line ./ 1000,
    label="Reference", linewidth=1, linestyle=:dash, color=:gray)

if !isnothing(gap)
    scatter!(pa, [gap.x1, gap.x2], [gap.G1, gap.G2] ./ 1000,
        markersize=6, color=:red, label="Equilibrium")
    plot!(pa, [gap.x1, gap.x2], [gap.G1, gap.G2] ./ 1000,
        linewidth=2, linestyle=:dash, color=:red, label="")
end

pa

In [ ]:
# Panel (b): Mixing Gibbs energy
pb = plot(x_Ag, G_mix,
    xlabel="x(Ag)", ylabel="G_mix [J/mol]",
    title="(b) Mixing Gibbs Energy",
    label="G_mix", linewidth=2, color=:blue,
    legend=:bottomright)

hline!(pb, [0], linestyle=:dash, color=:gray, label="")

if !isnothing(gap)
    vspan!(pb, [gap.x1, gap.x2], alpha=0.2, color=:red, label="Two-phase")
end

pb

In [ ]:
# Panel (c): Mixing enthalpy
pc = plot(x_Ag, H_mix,
    xlabel="x(Ag)", ylabel="H_mix [J/mol]",
    title="(c) Mixing Enthalpy",
    label="H_mix", linewidth=2, color=:red,
    legend=:bottomright)

hline!(pc, [0], linestyle=:dash, color=:gray, label="")

pc

In [ ]:
# Panel (d): Mixing entropy with ideal comparison
R = 8.314462618
S_ideal = -R .* (x_Ag .* log.(max.(x_Ag, 1e-10)) .+ x_Cu .* log.(max.(x_Cu, 1e-10)))

pd = plot(x_Ag, S_mix,
    xlabel="x(Ag)", ylabel="S_mix [J/(mol·K)]",
    title="(d) Mixing Entropy",
    label="S_mix", linewidth=2, color=:green,
    legend=:topright)

plot!(pd, x_Ag, S_ideal,
    label="S_ideal", linewidth=1, linestyle=:dash, color=:orange)

pd

In [ ]:
# Combined 2x2 panel
p_combined = plot(pa, pb, pc, pd,
    layout=(2, 2), size=(900, 700),
    plot_title="Ag-Cu FCC at T=$(Int(T))K",
    margin=5Plots.mm)

## 7. Interactive: Change Temperature

Try different temperatures to see how the miscibility gap changes.

In [ ]:
# Function to calculate and plot at different temperatures
function plot_at_temperature(T_new)
    scan_new = scan_composition(fcc, T_new, db, solver)
    gap_new = find_miscibility_gap(fcc, T_new, db)
    
    p = plot(scan_new.x_grid, scan_new.G_values ./ 1000,
        xlabel="x(Ag)", ylabel="G [kJ/mol]",
        title="Ag-Cu FCC at T=$(Int(T_new))K",
        label="G(x)", linewidth=2, color=:blue,
        legend=:topright, size=(600, 400))
    
    if !isnothing(gap_new)
        scatter!(p, [gap_new.x1, gap_new.x2], [gap_new.G1, gap_new.G2] ./ 1000,
            markersize=6, color=:red, label="x1=$(round(gap_new.x1, digits=3)), x2=$(round(gap_new.x2, digits=3))")
        plot!(p, [gap_new.x1, gap_new.x2], [gap_new.G1, gap_new.G2] ./ 1000,
            linewidth=2, linestyle=:dash, color=:red, label="")
    else
        annotate!(p, 0.5, minimum(scan_new.G_values)/1000 + 1, 
            text("No miscibility gap", 10, :center))
    end
    
    return p
end

# Example: T = 1200K
plot_at_temperature(1200.0)

In [ ]:
# Compare multiple temperatures
T_values = [900.0, 1000.0, 1200.0, 1400.0]
plots = [plot_at_temperature(T_i) for T_i in T_values]
plot(plots..., layout=(2,2), size=(1000, 800))

## Summary

This notebook demonstrated the thermodynamic property functions:

1. **`thermodynamic_properties(phase, T, y, db)`** - Efficient one-call calculation  
   Returns NamedTuple: `(G, H, S, G_mix, H_mix, S_mix)`

2. **Individual functions** (also available):
   - `calculate_mixing_gibbs(phase, T, y, db)`
   - `calculate_mixing_enthalpy(phase, T, y, db)`
   - `calculate_mixing_entropy(phase, T, y, db)`
   - `calculate_entropy(phase, T, y, db)`
   - `calculate_enthalpy(phase, T, y, db)`

These functions use automatic differentiation (ForwardDiff) for accurate temperature derivatives.